In [1]:
import pickle
import pandas as pd 
import numpy as np
from tqdm import tqdm
import random

In [2]:
positive_val = pd.read_csv("../data/filtered_deconvolved_positive_validation_set.csv")
positive_val

,epitope,label,MHC_expanded,peptide_length,key_binder,hla_sequence,predict_on,logit_1,logit_2,final_predictions
0,LSTVGPRL,1,C0304,8,537649,MRVMAPRTLILLLSGALALTETWAGSHSMRYFYTAVSRPGRGEPHF...,MRVMAPRTLILLLSGALALTETWAGSHSMRYFYTAVSRPGRGEPHF...,-1.635385,1.839033,0.862834
1,AETTTLFQF,1,B4403,9,317702,MRVTAPRTLLLLLWGAVALTETWAGSHSMRYFYTAMSRPGRGEPRF...,MRVTAPRTLLLLLWGAVALTETWAGSHSMRYFYTAMSRPGRGEPRF...,-3.080864,3.342883,0.965871
2,LPMKVRALGL,1,B0702,10,518233,MLVMAPRTVLLLLSAALALTETWAGSHSMRYFYTSVSRPGRGEPRF...,MLVMAPRTVLLLLSAALALTETWAGSHSMRYFYTSVSRPGRGEPRF...,-2.684186,2.932315,0.949421
3,DIQSSGRAK,1,A0301,9,12400,MAVMAPRTLLLLLSGALALTQTWAGSHSMRYFFTSVSRPGRGEPRF...,MAVMAPRTLLLLLSGALALTQTWAGSHSMRYFFTSVSRPGRGEPRF...,3.406410,-3.716567,0.023740
4,TMGHHTVGLK,1,A0301,10,308039,MAVMAPRTLLLLLSGALALTQTWAGSHSMRYFFTSVSRPGRGEPRF...,MAVMAPRTLLLLLSGALALTQTWAGSHSMRYFFTSVSRPGRGEPRF...,1.666246,-1.937590,0.125913
...,...,...,...,...,...,...,...,...,...,...
393817,HRFWKTELF,1,B2703,9,110624,MRVTAPRTLLLLLWGAVALTETWAGSHSMRYFHTSVSRPGRGEPRF...,MRVTAPRTLLLLLWGAVALTETWAGSHSMRYFHTSVSRPGRGEPRF...,-2.835380,3.089154,0.956443
393818,FPAPGKPGNYQY,1,B3501,12,159676,MRVTAPRTVLLLLWGAVALTETWAGSHSMRYFYTAMSRPGRGEPRF...,MRVTAPRTVLLLLWGAVALTETWAGSHSMRYFYTAMSRPGRGEPRF...,-2.815482,3.057637,0.955111
393819,RLMKHDVNL,1,A0201,9,519647,MAVMAPRTLVLLLSGALALTQTWAGSHSMRYFFTSVSRPGRGEPRF...,MAVMAPRTLVLLLSGALALTQTWAGSHSMRYFFTSVSRPGRGEPRF...,-1.801216,2.017691,0.882642
393820,ESLFVSNHAY,1,B1501,10,460875,MRVTAPRTVLLLLSGALALTETWAGSHSMRYFYTAMSRPGRGEPRF...,MRVTAPRTVLLLLSGALALTETWAGSHSMRYFYTAMSRPGRGEPRF...,-1.179181,1.330194,0.790873


In [3]:
#import negatives dict for negative sampling
negative_dict = "../data/negative_samples.pkl"
with open(negative_dict, "rb") as f:
    negative_dict = pickle.load(f)

negative_dict

{7: {'VQSFIQG',
  'QRCVPQD',
  'HIVDFKE',
  'TVVFGVF',
  'LEEAEKR',
  'DRSECKE',
  'MSLCGAR',
  'CCGNFSS',
  'VREEVID',
  'YSKDVLL',
  'EEDRMLT',
  'DAAAAEL',
  'NQQMCGG',
  'PYREALP',
  'PNFMVMQ',
  'RRIFQTQ',
  'ANNCPAL',
  'ADVNSGF',
  'TKHSKFN',
  'KETLVIM',
  'QRSQLQI',
  'HGRSYSL',
  'VLPAEVK',
  'QPSREVL',
  'SGALSGA',
  'GAGPGSP',
  'GGCGMVV',
  'GECFCRS',
  'NHSLDTD',
  'VVYNTEG',
  'PDFNSQF',
  'NNLANIK',
  'GKEWEEY',
  'CGGNKTA',
  'WLPLTDG',
  'SWPVAPR',
  'RSISALP',
  'VDIPHSS',
  'MDFSSME',
  'HIRISDL',
  'ELQLFLF',
  'DVRMIDF',
  'TNKVVSV',
  'ANKTRLP',
  'LAGLLLL',
  'APASRSH',
  'MEVEGEK',
  'SGKYTCR',
  'DQVETQG',
  'NIIDLLA',
  'LGHEWRL',
  'SNNIDVK',
  'PPATNEQ',
  'IGAGALY',
  'NLRKALL',
  'GWRHGPI',
  'CRIAART',
  'RNFMLAF',
  'LFSRWRT',
  'SGYFDER',
  'GAASFLI',
  'SNAGEYS',
  'QGCISDF',
  'VAVVVST',
  'LESEAEL',
  'SFTEVKV',
  'FDRAVVL',
  'QAPIPLT',
  'ELVTMMG',
  'FSVPKIP',
  'AQNKLKA',
  'STISSAP',
  'SEDTTVC',
  'PRSPAHL',
  'SRFNELP',
  'NQLMTTN',
  'VEVEPE

In [4]:
#Get one negative per positive from positive validation data
positive_val['Length'] = positive_val.epitope.apply(lambda x: len(x))

allele_mapper = pd.Series(list(positive_val['hla_sequence']), index = list(positive_val['MHC_expanded']))
allele_mapper.drop_duplicates(inplace=True)

negs = 1
#Create `negs` negatives for every positive
for i, allele in enumerate(tqdm(list(positive_val.MHC_expanded.drop_duplicates()))):
    sub_df = positive_val.query('MHC_expanded == @allele').copy()
    for length in list(sub_df['Length'].drop_duplicates()):
        sub_df_by_len = sub_df.query('Length == @length')
        use_int = negs * len(sub_df_by_len)
        add_these = pd.DataFrame({'MHC_expanded': [allele]*use_int,
                     'Length': [length]*use_int, 
                     'epitope': random.sample(negative_dict[length], use_int),
                     'label': [0]*use_int})
        positive_val = positive_val.append(add_these, ignore_index=True)

positive_val.reset_index(drop=True, inplace=True)

positive_val['hla_sequence'] = positive_val['MHC_expanded'].map(allele_mapper)
positive_val['predict_on'] = positive_val['hla_sequence'] + positive_val['epitope'] 
positive_val['predict_on_len'] = positive_val['predict_on'].apply(lambda x: len(x))

100%|█████████████████████████████████████████████████████████████████████████████████| 143/143 [03:49<00:00,  1.61s/it]


In [5]:
positive_val.loc[positive_val.label == 0]

,epitope,label,MHC_expanded,peptide_length,key_binder,hla_sequence,predict_on,logit_1,logit_2,final_predictions,Length,predict_on_len
393822,SSYRNDSS,0,C0304,NaN,NaN,MRVMAPRTLILLLSGALALTETWAGSHSMRYFYTAVSRPGRGEPHF...,MRVMAPRTLILLLSGALALTETWAGSHSMRYFYTAVSRPGRGEPHF...,NaN,NaN,NaN,8,374
393823,CSKCEKSF,0,C0304,NaN,NaN,MRVMAPRTLILLLSGALALTETWAGSHSMRYFYTAVSRPGRGEPHF...,MRVMAPRTLILLLSGALALTETWAGSHSMRYFYTAVSRPGRGEPHF...,NaN,NaN,NaN,8,374
393824,KCIQCFSP,0,C0304,NaN,NaN,MRVMAPRTLILLLSGALALTETWAGSHSMRYFYTAVSRPGRGEPHF...,MRVMAPRTLILLLSGALALTETWAGSHSMRYFYTAVSRPGRGEPHF...,NaN,NaN,NaN,8,374
393825,PDQRDEAP,0,C0304,NaN,NaN,MRVMAPRTLILLLSGALALTETWAGSHSMRYFYTAVSRPGRGEPHF...,MRVMAPRTLILLLSGALALTETWAGSHSMRYFYTAVSRPGRGEPHF...,NaN,NaN,NaN,8,374
393826,AHTTFLEQ,0,C0304,NaN,NaN,MRVMAPRTLILLLSGALALTETWAGSHSMRYFYTAVSRPGRGEPHF...,MRVMAPRTLILLLSGALALTETWAGSHSMRYFYTAVSRPGRGEPHF...,NaN,NaN,NaN,8,374
...,...,...,...,...,...,...,...,...,...,...,...,...
787639,KMKAKKGML,0,B5002,NaN,NaN,MRVTAPRTVLLLLSAALALTETWAGSHSMRYFHTAMSRPGRGEPRF...,MRVTAPRTVLLLLSAALALTETWAGSHSMRYFHTAMSRPGRGEPRF...,NaN,NaN,NaN,9,371
787640,ETAEGGQGL,0,B4406,NaN,NaN,MRVTAPRTLLLLLWGAVALTETWAGSHSMRYFYTAMSRPGRGEPRF...,MRVTAPRTLLLLLWGAVALTETWAGSHSMRYFYTAMSRPGRGEPRF...,NaN,NaN,NaN,9,371
787641,VNRPQIHKL,0,B4406,NaN,NaN,MRVTAPRTLLLLLWGAVALTETWAGSHSMRYFYTAMSRPGRGEPRF...,MRVTAPRTLLLLLWGAVALTETWAGSHSMRYFYTAMSRPGRGEPRF...,NaN,NaN,NaN,9,371
787642,APPRGRILG,0,B4406,NaN,NaN,MRVTAPRTLLLLLWGAVALTETWAGSHSMRYFYTAMSRPGRGEPRF...,MRVTAPRTLLLLLWGAVALTETWAGSHSMRYFYTAMSRPGRGEPRF...,NaN,NaN,NaN,9,371


In [6]:
positive_val.loc[(~(positive_val.epitope.isin(set(positive_val.loc[positive_val['label'] ==1, 'epitope']))))]

,epitope,label,MHC_expanded,peptide_length,key_binder,hla_sequence,predict_on,logit_1,logit_2,final_predictions,Length,predict_on_len
393822,SSYRNDSS,0,C0304,NaN,NaN,MRVMAPRTLILLLSGALALTETWAGSHSMRYFYTAVSRPGRGEPHF...,MRVMAPRTLILLLSGALALTETWAGSHSMRYFYTAVSRPGRGEPHF...,NaN,NaN,NaN,8,374
393823,CSKCEKSF,0,C0304,NaN,NaN,MRVMAPRTLILLLSGALALTETWAGSHSMRYFYTAVSRPGRGEPHF...,MRVMAPRTLILLLSGALALTETWAGSHSMRYFYTAVSRPGRGEPHF...,NaN,NaN,NaN,8,374
393824,KCIQCFSP,0,C0304,NaN,NaN,MRVMAPRTLILLLSGALALTETWAGSHSMRYFYTAVSRPGRGEPHF...,MRVMAPRTLILLLSGALALTETWAGSHSMRYFYTAVSRPGRGEPHF...,NaN,NaN,NaN,8,374
393825,PDQRDEAP,0,C0304,NaN,NaN,MRVMAPRTLILLLSGALALTETWAGSHSMRYFYTAVSRPGRGEPHF...,MRVMAPRTLILLLSGALALTETWAGSHSMRYFYTAVSRPGRGEPHF...,NaN,NaN,NaN,8,374
393826,AHTTFLEQ,0,C0304,NaN,NaN,MRVMAPRTLILLLSGALALTETWAGSHSMRYFYTAVSRPGRGEPHF...,MRVMAPRTLILLLSGALALTETWAGSHSMRYFYTAVSRPGRGEPHF...,NaN,NaN,NaN,8,374
...,...,...,...,...,...,...,...,...,...,...,...,...
787639,KMKAKKGML,0,B5002,NaN,NaN,MRVTAPRTVLLLLSAALALTETWAGSHSMRYFHTAMSRPGRGEPRF...,MRVTAPRTVLLLLSAALALTETWAGSHSMRYFHTAMSRPGRGEPRF...,NaN,NaN,NaN,9,371
787640,ETAEGGQGL,0,B4406,NaN,NaN,MRVTAPRTLLLLLWGAVALTETWAGSHSMRYFYTAMSRPGRGEPRF...,MRVTAPRTLLLLLWGAVALTETWAGSHSMRYFYTAMSRPGRGEPRF...,NaN,NaN,NaN,9,371
787641,VNRPQIHKL,0,B4406,NaN,NaN,MRVTAPRTLLLLLWGAVALTETWAGSHSMRYFYTAMSRPGRGEPRF...,MRVTAPRTLLLLLWGAVALTETWAGSHSMRYFYTAMSRPGRGEPRF...,NaN,NaN,NaN,9,371
787642,APPRGRILG,0,B4406,NaN,NaN,MRVTAPRTLLLLLWGAVALTETWAGSHSMRYFYTAMSRPGRGEPRF...,MRVTAPRTLLLLLWGAVALTETWAGSHSMRYFYTAMSRPGRGEPRF...,NaN,NaN,NaN,9,371


In [7]:
negatives_df = positive_val.loc[positive_val.label == 0].copy()
negatives_df.sort_values(['MHC_expanded', 'Length'], inplace=True)
negatives_df.to_csv("../data/negatives_validation_set.csv", index=False)

In [8]:
#Tokenize
from transformers import AutoTokenizer

del positive_val 
del negative_dict

model_checkpoint = "esm2_t12_35M_UR50D_MHCI_classification/checkpoint-310935/"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

test_sequences = negatives_df['predict_on'].tolist()
test_labels = negatives_df['label'].tolist()


#print("Tokenizing validation data...")
test_tokenized = tokenizer(test_sequences)
with open (f"../data/MA_dsets/validation_negatives.pkl", 'wb') as f:
    pickle.dump([negatives_df, test_tokenized, test_labels], f)

In [13]:
from datasets import Dataset

val_dset = Dataset.from_dict(test_tokenized)
val_dset = val_dset.add_column("labels", test_labels)
val_dset

Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 393822
})

In [15]:
with open("../data/val_dset_negs.pkl", 'wb') as f:
    pickle.dump(val_dset, f)

In [17]:
from tqdm import tqdm
import tensorflow as tf
import torch
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer


model_checkpoint = "esm2_t12_35M_UR50D_MHCI_classification/checkpoint-310935/"
device = torch.device('cuda')
model = AutoModelForSequenceClassification.from_pretrained(model_checkpoint, num_labels=2).to(device)

#model.load_state_dict(checkpoint['model_state_dict'])
#optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
#epoch = checkpoint['epoch']
#loss = checkpoint['loss']



model.eval(); #enable for prediction
final_predictions = []
final_logits = []

with torch.no_grad():
    iterator = 64 #Number of predictions to get with each loop
    for i, n in enumerate(tqdm(range(0, val_dset.num_rows, iterator))):
        try:
            # Retrieve item
            item = val_dset[n:n+iterator]
            sequence = item['input_ids']
            label = item['labels']
            attention_mask = item['attention_mask']
    
            # Generate prediction
            predictions = model(torch.IntTensor(sequence).to(device), torch.IntTensor(attention_mask).to(device))
            final_logits = final_logits + [x for x in predictions['logits'].cpu()]
            ret = [x[1].numpy() for x in tf.nn.sigmoid(predictions['logits'].cpu())]
            final_predictions = final_predictions + ret
        except: #Different lengths occurred... Drat
            for index in range(n, n+iterator):
                item = val_dset[index]
                sequence = [item['input_ids']]
                label = [item['labels']]
                attention_mask = [item['attention_mask']]

                # Generate prediction
                predictions = model(torch.IntTensor(sequence).to(device), torch.IntTensor(attention_mask).to(device))
                final_logits = final_logits + [x for x in predictions['logits'].cpu()]
                ret = [x[1].numpy() for x in tf.nn.sigmoid(predictions['logits'].cpu())]
                final_predictions = final_predictions + ret  
                

        # Predicted class value using argmax
        # predicted_class = np.argmax(prediction)
    
 

  0%|                                                                                          | 0/6154 [00:00<?, ?it/s]2024-07-30 13:31:34.864856: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] could not open file to read NUMA node: /sys/bus/pci/devices/0000:0c:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-07-30 13:31:34.865168: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1960] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...
100%|███████████████████████████████████████████████████████████████████████████████| 6154/6154 [34:48<00:00,  2.95it/s]


In [19]:
final_logits

[tensor([ 4.0310, -4.3295]),
 tensor([ 2.8921, -3.2051]),
 tensor([ 3.6879, -3.9914]),
 tensor([ 3.5957, -3.9099]),
 tensor([ 3.1443, -3.4444]),
 tensor([ 4.2365, -4.5107]),
 tensor([ 3.6854, -3.9781]),
 tensor([ 3.6098, -3.9126]),
 tensor([ 2.4043, -2.7176]),
 tensor([ 4.2078, -4.4732]),
 tensor([ 2.6289, -2.8982]),
 tensor([ 3.3893, -3.7077]),
 tensor([ 4.4148, -4.6574]),
 tensor([ 3.4674, -3.7429]),
 tensor([-1.1612,  1.3622]),
 tensor([ 3.0165, -3.3377]),
 tensor([ 3.8737, -4.1743]),
 tensor([ 4.3056, -4.5780]),
 tensor([ 3.9070, -4.2078]),
 tensor([ 3.6617, -3.9810]),
 tensor([ 3.4730, -3.7759]),
 tensor([ 3.7164, -4.0301]),
 tensor([ 2.7155, -3.0064]),
 tensor([ 2.4996, -2.8202]),
 tensor([ 4.2442, -4.5145]),
 tensor([ 3.6022, -3.9051]),
 tensor([ 2.9542, -3.2655]),
 tensor([ 3.4000, -3.7083]),
 tensor([ 3.7150, -4.0103]),
 tensor([ 3.0421, -3.3444]),
 tensor([ 3.2196, -3.5351]),
 tensor([ 3.8756, -4.1615]),
 tensor([ 4.0608, -4.3473]),
 tensor([ 4.3472, -4.6081]),
 tensor([ 4.40

In [20]:
with open("../data/val_dset_negs_logits.pkl", 'wb') as f:
     pickle.dump([val_dset, final_logits], f)

In [24]:
negatives_df['logit_1'] = [x[0].numpy() for x in final_logits]
negatives_df['logit_2'] = [x[1].numpy() for x in final_logits]
negatives_df['final_predictions'] = final_predictions
negatives_df

,epitope,label,MHC_expanded,peptide_length,key_binder,hla_sequence,predict_on,logit_1,logit_2,final_predictions,Length,predict_on_len
608414,NLVRVLQG,0,A0101,NaN,NaN,MAVMAPRTLLLLLSGALALTQTWAGSHSMRYFFTSVSRPGRGEPRF...,MAVMAPRTLLLLLSGALALTQTWAGSHSMRYFFTSVSRPGRGEPRF...,4.0310116,-4.329536,0.013002,8,373
608415,IVNRINTL,0,A0101,NaN,NaN,MAVMAPRTLLLLLSGALALTQTWAGSHSMRYFFTSVSRPGRGEPRF...,MAVMAPRTLLLLLSGALALTQTWAGSHSMRYFFTSVSRPGRGEPRF...,2.8920715,-3.2050636,0.038976,8,373
608416,GLSDPAHV,0,A0101,NaN,NaN,MAVMAPRTLLLLLSGALALTQTWAGSHSMRYFFTSVSRPGRGEPRF...,MAVMAPRTLLLLLSGALALTQTWAGSHSMRYFFTSVSRPGRGEPRF...,3.6878822,-3.9913788,0.018139,8,373
608417,RDPNSAVV,0,A0101,NaN,NaN,MAVMAPRTLLLLLSGALALTQTWAGSHSMRYFFTSVSRPGRGEPRF...,MAVMAPRTLLLLLSGALALTQTWAGSHSMRYFFTSVSRPGRGEPRF...,3.5957105,-3.9098763,0.019649,8,373
608418,YASKLGDD,0,A0101,NaN,NaN,MAVMAPRTLLLLLSGALALTQTWAGSHSMRYFFTSVSRPGRGEPRF...,MAVMAPRTLLLLLSGALALTQTWAGSHSMRYFFTSVSRPGRGEPRF...,3.144347,-3.4443574,0.030938,8,373
...,...,...,...,...,...,...,...,...,...,...,...,...
507669,YILPILGICNE,0,C1701,NaN,NaN,MRVMAPQALLLLLSGALALIETWAGSHSMRYFYTAVSRPGRGEPRF...,MRVMAPQALLLLLSGALALIETWAGSHSMRYFYTAVSRPGRGEPRF...,2.986101,-3.2626731,0.036874,11,383
507670,SAPEEGEVSDS,0,C1701,NaN,NaN,MRVMAPQALLLLLSGALALIETWAGSHSMRYFYTAVSRPGRGEPRF...,MRVMAPQALLLLLSGALALIETWAGSHSMRYFYTAVSRPGRGEPRF...,3.5382085,-3.7944148,0.022001,11,383
507671,GLAIVTKYITK,0,C1701,NaN,NaN,MRVMAPQALLLLLSGALALIETWAGSHSMRYFYTAVSRPGRGEPRF...,MRVMAPQALLLLLSGALALIETWAGSHSMRYFYTAVSRPGRGEPRF...,2.7159486,-2.995777,0.047617,11,383
507672,SKKVDFELAVQ,0,C1701,NaN,NaN,MRVMAPQALLLLLSGALALIETWAGSHSMRYFYTAVSRPGRGEPRF...,MRVMAPQALLLLLSGALALIETWAGSHSMRYFYTAVSRPGRGEPRF...,3.2676017,-3.5625355,0.027584,11,383


In [25]:
negatives_df.reset_index(drop=True, inplace=True)
negatives_df.to_csv("../data/filtered_deconvolved_negative_validation_set.csv", index=False)

In [26]:
positives_df = pd.read_csv("../data/filtered_deconvolved_positive_validation_set.csv")
positives_df

,epitope,label,MHC_expanded,peptide_length,key_binder,hla_sequence,predict_on,logit_1,logit_2,final_predictions
0,LSTVGPRL,1,C0304,8,537649,MRVMAPRTLILLLSGALALTETWAGSHSMRYFYTAVSRPGRGEPHF...,MRVMAPRTLILLLSGALALTETWAGSHSMRYFYTAVSRPGRGEPHF...,-1.635385,1.839033,0.862834
1,AETTTLFQF,1,B4403,9,317702,MRVTAPRTLLLLLWGAVALTETWAGSHSMRYFYTAMSRPGRGEPRF...,MRVTAPRTLLLLLWGAVALTETWAGSHSMRYFYTAMSRPGRGEPRF...,-3.080864,3.342883,0.965871
2,LPMKVRALGL,1,B0702,10,518233,MLVMAPRTVLLLLSAALALTETWAGSHSMRYFYTSVSRPGRGEPRF...,MLVMAPRTVLLLLSAALALTETWAGSHSMRYFYTSVSRPGRGEPRF...,-2.684186,2.932315,0.949421
3,DIQSSGRAK,1,A0301,9,12400,MAVMAPRTLLLLLSGALALTQTWAGSHSMRYFFTSVSRPGRGEPRF...,MAVMAPRTLLLLLSGALALTQTWAGSHSMRYFFTSVSRPGRGEPRF...,3.406410,-3.716567,0.023740
4,TMGHHTVGLK,1,A0301,10,308039,MAVMAPRTLLLLLSGALALTQTWAGSHSMRYFFTSVSRPGRGEPRF...,MAVMAPRTLLLLLSGALALTQTWAGSHSMRYFFTSVSRPGRGEPRF...,1.666246,-1.937590,0.125913
...,...,...,...,...,...,...,...,...,...,...
393817,HRFWKTELF,1,B2703,9,110624,MRVTAPRTLLLLLWGAVALTETWAGSHSMRYFHTSVSRPGRGEPRF...,MRVTAPRTLLLLLWGAVALTETWAGSHSMRYFHTSVSRPGRGEPRF...,-2.835380,3.089154,0.956443
393818,FPAPGKPGNYQY,1,B3501,12,159676,MRVTAPRTVLLLLWGAVALTETWAGSHSMRYFYTAMSRPGRGEPRF...,MRVTAPRTVLLLLWGAVALTETWAGSHSMRYFYTAMSRPGRGEPRF...,-2.815482,3.057637,0.955111
393819,RLMKHDVNL,1,A0201,9,519647,MAVMAPRTLVLLLSGALALTQTWAGSHSMRYFFTSVSRPGRGEPRF...,MAVMAPRTLVLLLSGALALTQTWAGSHSMRYFFTSVSRPGRGEPRF...,-1.801216,2.017691,0.882642
393820,ESLFVSNHAY,1,B1501,10,460875,MRVTAPRTVLLLLSGALALTETWAGSHSMRYFYTAMSRPGRGEPRF...,MRVTAPRTVLLLLSGALALTETWAGSHSMRYFYTAMSRPGRGEPRF...,-1.179181,1.330194,0.790873


In [27]:
combined_df = pd.concat([positives_df, negatives_df]).copy()
combined_df.reset_index(drop=True, inplace=True)
combined_df.to_csv("../data/filtered_deconvolved_final_validation_set.csv", index=False)
combined_df

,epitope,label,MHC_expanded,peptide_length,key_binder,hla_sequence,predict_on,logit_1,logit_2,final_predictions,Length,predict_on_len
0,LSTVGPRL,1,C0304,8.0,537649.0,MRVMAPRTLILLLSGALALTETWAGSHSMRYFYTAVSRPGRGEPHF...,MRVMAPRTLILLLSGALALTETWAGSHSMRYFYTAVSRPGRGEPHF...,-1.635385,1.839033,0.862834,NaN,NaN
1,AETTTLFQF,1,B4403,9.0,317702.0,MRVTAPRTLLLLLWGAVALTETWAGSHSMRYFYTAMSRPGRGEPRF...,MRVTAPRTLLLLLWGAVALTETWAGSHSMRYFYTAMSRPGRGEPRF...,-3.080864,3.342883,0.965871,NaN,NaN
2,LPMKVRALGL,1,B0702,10.0,518233.0,MLVMAPRTVLLLLSAALALTETWAGSHSMRYFYTSVSRPGRGEPRF...,MLVMAPRTVLLLLSAALALTETWAGSHSMRYFYTSVSRPGRGEPRF...,-2.684186,2.932315,0.949421,NaN,NaN
3,DIQSSGRAK,1,A0301,9.0,12400.0,MAVMAPRTLLLLLSGALALTQTWAGSHSMRYFFTSVSRPGRGEPRF...,MAVMAPRTLLLLLSGALALTQTWAGSHSMRYFFTSVSRPGRGEPRF...,3.40641,-3.716567,0.023740,NaN,NaN
4,TMGHHTVGLK,1,A0301,10.0,308039.0,MAVMAPRTLLLLLSGALALTQTWAGSHSMRYFFTSVSRPGRGEPRF...,MAVMAPRTLLLLLSGALALTQTWAGSHSMRYFFTSVSRPGRGEPRF...,1.666246,-1.93759,0.125913,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
787639,YILPILGICNE,0,C1701,NaN,NaN,MRVMAPQALLLLLSGALALIETWAGSHSMRYFYTAVSRPGRGEPRF...,MRVMAPQALLLLLSGALALIETWAGSHSMRYFYTAVSRPGRGEPRF...,2.986101,-3.2626731,0.036874,11.0,383.0
787640,SAPEEGEVSDS,0,C1701,NaN,NaN,MRVMAPQALLLLLSGALALIETWAGSHSMRYFYTAVSRPGRGEPRF...,MRVMAPQALLLLLSGALALIETWAGSHSMRYFYTAVSRPGRGEPRF...,3.5382085,-3.7944148,0.022001,11.0,383.0
787641,GLAIVTKYITK,0,C1701,NaN,NaN,MRVMAPQALLLLLSGALALIETWAGSHSMRYFYTAVSRPGRGEPRF...,MRVMAPQALLLLLSGALALIETWAGSHSMRYFYTAVSRPGRGEPRF...,2.7159486,-2.995777,0.047617,11.0,383.0
787642,SKKVDFELAVQ,0,C1701,NaN,NaN,MRVMAPQALLLLLSGALALIETWAGSHSMRYFYTAVSRPGRGEPRF...,MRVMAPQALLLLLSGALALIETWAGSHSMRYFYTAVSRPGRGEPRF...,3.2676017,-3.5625355,0.027584,11.0,383.0
